In [1]:
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates
import datetime
import matplotlib.dates as mdates
from datetime import date
import threading
import time
import pandas

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
		self.data = [] #Initialize variable to store candle

	def historicalData(self, reqId, bar):
		self.data.append([bar.date, bar.open, bar.close, bar.volume])
		print(bar)
		
def run_loop():
	app.run()

app = IBapi()
app.connect('127.0.0.1', 7496, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
contract = Contract()
contract.symbol = 'QQQ'
contract.secType = 'STK'
contract.exchange = 'SMART'
contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, contract, '', '6 M', '5 mins', 'TRADES', 1, 2, False, [])

time.sleep(59) #sleep to allow enough time for data to be returned

df = pandas.DataFrame(app.data, columns=['DateTime', 'Open', 'Close', 'Volume'])
df['DateTime'] = pandas.to_datetime(df['DateTime'],unit='s').dt.tz_localize(tz='UTC')
df['DateTime'] = df['DateTime'].dt.tz_convert(tz='US/Eastern')
df.to_csv('QqqHist.csv')  

print(df)


app.disconnect()

ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefnj


Date: 1682947800, Open: 322.09, High: 322.36, Low: 321.92, Close: 322.13, Volume: 789082, WAP: 322.133, BarCount: 3942
Date: 1682948100, Open: 322.12, High: 322.3, Low: 321.75, Close: 322.29, Volume: 630057, WAP: 322.015, BarCount: 3566
Date: 1682948400, Open: 322.27, High: 322.49, Low: 322.03, Close: 322.1, Volume: 499475, WAP: 322.286, BarCount: 2683
Date: 1682948700, Open: 322.09, High: 322.5, Low: 321.95, Close: 322.46, Volume: 538842, WAP: 322.279, BarCount: 2655
Date: 1682949000, Open: 322.47, High: 322.77, Low: 322.3, Close: 322.58, Volume: 564619, WAP: 322.575, BarCount: 3128
Date: 1682949300, Open: 322.57, High: 322.62, Low: 322.22, Close: 322.35, Volume: 615190, WAP: 322.38, BarCount: 3355
Date: 1682949600, Open: 322.34, High: 322.34, Low: 321.7, Close: 321.96, Volume: 862732, WAP: 322.001, BarCount: 4748
Date: 1682949900, Open: 321.97, High: 322.23, Low: 321.76, Close: 322.22, Volume: 596711, WAP: 321.992, BarCount: 3071
Date: 1682950200, Open: 322.22, High: 322.32, Low: 321

In [ ]:
#For fun, an average graph. averages all the differences of all the days and shows it in a graph. Objective is to show the average daily movement patterns

dfAverage = pandas.DataFrame(columns=['Average','Time'])
currentAve = 0
count = len(df.date.unique())

for currentTime in df.time.unique():
    average = 0
    for currentDate in df.date.unique():
        if(df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)].empty is False):
            average = average + df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)]['Close'].values[0]-df.loc[(df['time'] == currentTime) & (df['date'] == currentDate)]['Open'].values[0]
        else:
            print(currentDate)
    currentAve = currentAve + (average/count)
    dfAverage.loc[len(dfAverage.index)] = [currentAve,currentTime] 

datetimes = [datetime.datetime.combine(datetime.date.today(), t) for t in dfAverage["Time"]]
plt.plot(datetimes, dfAverage["Average"])
fmt = mdates.DateFormatter('%H:%M')
plt.gca().xaxis.set_major_formatter(fmt)
